In [2]:
import hashlib
from binascii import b2a_hex
import base64
from Crypto.Cipher import AES
import sys
import time
import json
import random

### Configuration

In [3]:
# test data ===============================================

user_id = "1493673206"
user_name = "felixlukk"

playlist_id = "2277932638"

# ranklist setting ----------------------------

# last week
rank_type_week = 1
# all time
rank_type_all = 0
# last week+all time
rank_type_week_all = -1

# default
rank_type = 1

# max song number
week_rank_max = 100

# max song number
all_rank_max = 100

# max song number
rank_max = all_rank_max


# playlist setting ------------------------------

# user favourite --------------------

# whether to crawl "favourite" list
is_playlists_default = True

# playlist type
normal_playlist = 4
# favourite
default_playlist = 5
# created
created_playlist = 6
# collected
collected_playlist = 7

# defalut
playlist_type = 4


# created playlist -------------------------

# whether to crawl "created" list
is_playlists_created = True

# max number
created_playlists_max = sys.maxsize

# collected playlist -------------------------

# whether to crawl "collected" list
is_playlists_collected = True

# max playlist number
collected_playlists_max = 5

# max songs
collected_songs_max = sys.maxsize



# basic request setting=================================================

# basic information
base_url = "http://music.163.com/"
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
host = "music.163.com"
language = "zh-CN,zh;q=0.9,en;q=0.8"
accept = "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8"

# basic header
user_headers = {
    "User-Agent": user_agent,
    "Host": host,
    "Accept": accept,
    "Referer": base_url,
    "Accept-Language": language
}

# request for eapi
user_headers_eapi = {
    'os': 'osx',
    'appver': '2.5.3',
    'requestId': str(random.randint(10000000, 99999999)),
    'clientSign': '',
}

# eapi common params
eapi_params = {
    'verifyId': 1,
    'os': 'OSX',
    'header': json.dumps(user_headers_eapi, separators=(',', ':'))
}


# AES encrypted================================================================


# total aes
aes_total = True

# offset params
aes_offset = 0

# limit number
aes_limit = 1000

aes_ranklist_type = -1


# api type
api_weapi = 0
api_eapi = 1
api_type = api_weapi

second_param = "010001"
third_param = "00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7"
forth_param = "0CoJUm6Qyw8W8jud"

# eapikey
eapikey = "e82ckenh8dichen8"

# for twic AES params
first_key = forth_param
second_key = 16 * 'F'

# offset
iv = "0102030405060708"

# encSecKey
encSecKey = "257348aecb5e556c066de214e531faadd1c55d814f9be95fd06d6bff9f4c7a41f831f6394d5a3fd2e3881736d94a02ca919d952872e7d0a50ebfa1769a7a62d512f5f1ca21aec60bc3819a9c3ffca5eca9a0dba6d6f7249b06f5965ecfff3695b54e1c28f3f624750ed39e7de08fc8493242e26dbc4484a01c76f739e135637c"

# url ================================================================

# user ranklist url
url_user_rank = "https://music.163.com/weapi/v1/play/record"

# user playlist url
url_user_playlists = "https://music.163.com/weapi/user/playlist"


### Encrypt request

In [4]:
# AES encrypt

def __AES_encrypt( mode, param, key, iv):
        """
        AES加密过程

        :param mode: AES模式
        :param param: AES加密对象
        :param key: AES加密秘钥
        :param iv: AES秘钥偏移量
        :return: AES加密结果
        """
        # cannot use len(param),beacuse it cannoot use chinese，so transfer to unicode first
        pad = 16 - len(param.encode()) % 16
        param = param + pad * chr(pad)
        encryptor = AES.new(key, mode, iv)
        encrypt_text = encryptor.encrypt(param)
        return encrypt_text

In [5]:
# # once encrypted params
def __get_eapi_params( eapi_url, first_param):

        first_param = first_param.encode()
        eapi_url = eapi_url.encode()
        params = b'nobody' + eapi_url + b'use' + first_param + b'md5forencrypt'
        md5 = hashlib.md5()
        md5.update(params)
        params = eapi_url + b'-36cd479b6b5-' + first_param + b'-36cd479b6b5-' + md5.hexdigest().encode()
        pad = 16 - len(params) % 16
        params = params + bytearray([pad] * pad)
        crypt = AES.new(eapikey, AES.MODE_ECB)
        params = crypt.encrypt(params)
        return b2a_hex(params).upper()


In [6]:
##twice encrypted params
def __get_weapi_params( first_param):
        
        params = __AES_encrypt(mode=AES.MODE_CBC, param=first_param, key=first_key, iv=iv)
        params = str(base64.b64encode(params))[2:-1]
        params = __AES_encrypt(mode=AES.MODE_CBC, param=params, key=second_key, iv=iv)
        return str(base64.b64encode(params))[2:-1]

In [7]:
# get encSecKey
def __get_encSecKey():
        return encSecKey

In [8]:
def get_form_data( first_param, api_type=api_weapi, eapi_url=None):
        """
        获取请求参数

        :param first_param: 第一个请求字典
        :param api_type: api类型
        :param eapi_url: eapi的请求链接
        :return:
        """
        if api_type == api_weapi:
            return {
                "params": __get_weapi_params(first_param),
                "encSecKey": __get_encSecKey()
            }
        elif api_type == api_eapi:
            return {
                "params": __get_eapi_params(eapi_url=eapi_url, first_param=first_param)
            }
        else:
            return None

In [9]:
#test
print(get_form_data("""{{uid:"{}",offset:"{}",total:"{}",limit:"{}",type:"{}",csrf_token:"{}"}}""" \
                          .format(user_id, 0, aes_total, aes_limit, aes_ranklist_type, "")))

{'params': 'e1DsKnuam7sZ3apwcfNbC44mvapDylFh+jqvtsAEEUW0XoyEPVhZyAFm+6CIN023PSoMolCi0MD/1rANUX+HH856BnCwduDooZENLuWKWP8IOOB6sKxk1L/FQDVVYMhz+LjbwDeQMkzq+R+YeIkhrw==', 'encSecKey': '257348aecb5e556c066de214e531faadd1c55d814f9be95fd06d6bff9f4c7a41f831f6394d5a3fd2e3881736d94a02ca919d952872e7d0a50ebfa1769a7a62d512f5f1ca21aec60bc3819a9c3ffca5eca9a0dba6d6f7249b06f5965ecfff3695b54e1c28f3f624750ed39e7de08fc8493242e26dbc4484a01c76f739e135637c'}


## Construct params

In [10]:
# ranklist params, id+rank_type
def get_first_param_ranklist(user_id=user_id, rank_type=rank_type):
        return True, """{{uid:"{}",type:"{}"}}""".format(user_id, rank_type)


In [11]:
#  id+ limit:maxnum of list song + offset
def get_first_param_user_playlists(user_id=user_id, limit=aes_limit, offset=aes_offset):
        return True, """{{uid:"{}",limit:"{}",offset:"{}"}}""".format(user_id, limit, offset)

In [12]:
# playlist_id + n:max_num
def get_first_param_playlist(playlist_id=playlist_id, n=100000, s=0):
        return True, """{{id:"{}",n:"{}",s:"{}"}}""".format(playlist_id, n, s)

In [13]:
print(get_first_param_ranklist())

(True, '{uid:"1493673206",type:"1"}')


In [14]:
print(get_first_param_user_playlists())

(True, '{uid:"1493673206",limit:"1000",offset:"0"}')


In [15]:
print(get_first_param_playlist())

(True, '{id:"2277932638",n:"100000",s:"0"}')


### Request data

In [16]:
import requests

In [27]:
def get_request_data(first_param=None, url=None, proxies=None, api_type=api_weapi, eapi_url=None):
        _form_data = None
        if first_param is not None:
            _form_data = get_form_data(first_param=first_param, api_type=api_type, eapi_url=eapi_url)
#         print(_form_data)
        try:
            response = requests.post(url=url, headers=user_headers, data=_form_data, proxies=proxies).content
            status = True
            print("get_request_data success",
                         "url:{} ,first_param:{},proxies:{}".format(url, first_param, proxies))
        except Exception as e:
            response = None
            status = False
            print("get_request_data failed",
                         "url:{} ,first_param:{},proxies:{},error_type:{},error:{}".format(url, first_param, proxies, type(e),e))
        return status, response


In [18]:
print(get_request_data())

None
get_request_data failed url:None ,first_param:None,proxies:None,error_type:<class 'requests.exceptions.MissingSchema'>,error:Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
(False, None)


### crawl playlist

In [19]:
# param: user_id + created_playlists_max: 用户创建的歌单获取最大数+collected_playlists_max: 用户收集的歌单获取最大数
# is_playlists_default: 是否爬取”用户喜欢的音乐“歌单 + aram is_playlists_created: 是否爬取用户创建的歌单
# is_playlists_collected: 是否爬取用户收藏的歌单
#   
def get_user_playlists(user_id=user_id, created_playlists_max=created_playlists_max,
                           collected_playlists_max=collected_playlists_max,
                           is_playlists_default=is_playlists_default,
                           is_playlists_created=is_playlists_created,
                           is_playlists_collected=is_playlists_collected):

        # request params ----------------------------------------------------
        _first_param = get_first_param_user_playlists(user_id=user_id)
        # data ----------------------------------------------------
        try:
            content = get_request_data(_first_param[1], url=url_user_playlists)
            if content[0]:
                json_playlists_data = json.loads(content[1])["playlist"]
            else:
                return False, None
        except Exception as e:
            print("get_user_playlists get failed",
                         "user_id:{},error_type:{},error:{}".format(user_id, type(e), e))
            return False, None
        # paraser data ----------------------------------------------------
        playlist_count = 0
        created_playlists_count = 0
        collected_playlists_count = 0
        playlist_list = []
        user_playlists_list = []
        tag_list = []
        playlist_tag_list = []
        try:
            while playlist_count < len(json_playlists_data):
                # favourite list
                if is_playlists_default:
                    parse = __parser(user_id=user_id, playlist_count=playlist_count, data=json_playlists_data,
                                          playlist_type=default_playlist)
                    playlist_list.append(parse["playlist_list"])
                    user_playlists_list.append(parse["user_playlist_list"])
                    if len(parse["tag_list"]) != 0:
                        tag_list.extend(parse["tag_list"])
                        playlist_tag_list.extend(parse["playlist_tag_list"])
                    playlist_count += 1
                    is_playlists_default = False
                    continue

                # created list
                elif is_playlists_created:
                    # if playlist= 0, means no created list, so skip
                    if playlist_count == 0:
                        playlist_count += 1
                        continue
                    # whether created 
                    elif str(json_playlists_data[playlist_count]["creator"]['userId']) == str(user_id):
                        if created_playlists_count < created_playlists_max:
                            parse = __parser(user_id=user_id, playlist_count=playlist_count,
                                                  data=json_playlists_data,
                                                  playlist_type=created_playlist)
                            playlist_list.append(parse["playlist_list"])
                            user_playlists_list.append(parse["user_playlist_list"])
                            if len(parse["tag_list"]) != 0:
                                tag_list.extend(parse["tag_list"])
                                playlist_tag_list.extend(parse["playlist_tag_list"])
                            created_playlists_count += 1
                        playlist_count += 1
                    else:
                        is_playlists_created = False
                    continue

                # collected playlist
                elif is_playlists_collected:
                    # whether collected
                    if str(json_playlists_data[playlist_count]["creator"]['userId']) != str(user_id):
                        if collected_playlists_count < collected_playlists_max:
                            parse = __parser(user_id=user_id, playlist_count=playlist_count,
                                                  data=json_playlists_data,
                                                  playlist_type=collected_playlist)
                            playlist_list.append(parse["playlist_list"])
                            user_playlists_list.append(parse["user_playlist_list"])
                            if len(parse["tag_list"]) != 0:
                                tag_list.extend(parse["tag_list"])
                                playlist_tag_list.extend(parse["playlist_tag_list"])
                            collected_playlists_count += 1
                    playlist_count += 1
                    continue
                break
        except Exception as e:
            print("get_user_playlists parse failed",
                         "user_id:{},created_playlists_count:{},collected_playlists_count:{},error_type:{},error:{}"
                         .format(user_id, created_playlists_count, collected_playlists_count, type(e), e))
            return False, None

        print("get user_playlist success",
                    "user_id:{},created_playlists_count:{},collected_playlists_count:{}"
                    .format(user_id, created_playlists_count, collected_playlists_count))
        return playlist_list,playlist_tag_list

In [20]:
#params user_id: 用户id + playlist_count: 歌单计数器 + param data: 待解析数据 + param playlist_type: 歌单类型

def __parser(user_id, playlist_count, data, playlist_type):
        
        tag_list = []
        playlist_tag_list = []
        for ptag in data[playlist_count]["tags"]:
            tag_list.append([ptag])
            playlist_tag_list.append([
                data[playlist_count]["id"],
                ptag
            ])
        return {
            "playlist_list": [
                data[playlist_count]["id"],
                data[playlist_count]["name"],
                data[playlist_count]["trackCount"],
                data[playlist_count]["playCount"],
                data[playlist_count]["updateTime"]
            ],
            "user_playlist_list": [
                user_id,
                data[playlist_count]["id"],
                playlist_type
            ],
            "tag_list": tag_list,
            "playlist_tag_list": playlist_tag_list
        }


In [28]:
print(get_user_playlists(user_id, created_playlists_max=5, collected_playlists_max=50,
                                             is_playlists_default=True, is_playlists_created=True,
                                              is_playlists_collected=True))

b'{"more":false,"playlist":[{"subscribers":[],"subscribed":false,"creator":{"defaultAvatar":false,"province":1000000,"authStatus":0,"followed":false,"avatarUrl":"http://p1.music.126.net/BGbjPEqVVcVz1-Lm8__Dlg==/109951163376705722.jpg","accountStatus":0,"gender":1,"city":1004400,"birthday":-2209017600000,"userId":1493673206,"userType":0,"nickname":"felixlukk","signature":"","description":"","detailDescription":"","avatarImgId":109951163376705722,"backgroundImgId":109951162868126486,"backgroundUrl":"http://p1.music.126.net/_f8R60U9mZ42sSNvdPn2sQ==/109951162868126486.jpg","authority":0,"mutual":false,"expertTags":null,"experts":null,"djStatus":0,"vipType":11,"remarkName":null,"avatarImgIdStr":"109951163376705722","backgroundImgIdStr":"109951162868126486","avatarImgId_str":"109951163376705722"},"artists":null,"tracks":null,"updateFrequency":null,"backgroundCoverId":0,"backgroundCoverUrl":null,"titleImage":0,"titleImageUrl":null,"englishTitle":null,"opRecommend":false,"recommendInfo":null,"

In [35]:
def get_user_ranklist_songs(user_id=user_id, rank_type=rank_type, rank_max=rank_max):

        _first_param = get_first_param_ranklist(user_id=user_id, rank_type=rank_type)
        json_data = None
        try:
            content = get_request_data(first_param=_first_param[1], url=url_user_rank)
            if content[0]:
                if rank_type == rank_type_all:
                    json_data = json.loads(content[1])["allData"]
                elif rank_type == rank_type_week:
                    json_data = json.loads(content[1])["weekData"]
            else:
                return False, None
        except KeyError as e:
            print(
                "get_user_ranklist_songs get failed, Maybe the guy's ranklist is hidden,can you see it in the webpage ?",
                "user_id:{},rank_type:{},ranklist_url:https://music.163.com/#/user/songs/rank?id={} ,error_type:{},error:{}"
                    .format(user_id, rank_type, user_id, type(e), e))
            return False, None
        except Exception as e:
            print("get_user_ranklist_songs get failed", "user_id:{},rank_type:{},error_type:{},error:{}"
                         .format(user_id, rank_type, type(e), e))
            return False, None
        ranklist_id = str(user_id) + "r" + str(rank_type)
        song_success_count = 0
        rank_list = []
        user_rank_list = []
        song_list = []
        artist_list = []
        artist_song_list = []
        song_rank_list = []
        user_song_list = []
        try:
            # ranklist_id rank_type rank_date
            rank_list.append([
                ranklist_id,
                rank_type,
                int(round(time.time() * 1000))
            ])
            # user_id ranklist_id
            user_rank_list.append([
                user_id,
                ranklist_id
            ])
            while song_success_count < rank_max and song_success_count < len(json_data):
                song_id = json_data[song_success_count]["song"]["id"]
                song_score = json_data[song_success_count]["score"]
#                 print(json_data)
#                 print('#############################')
#                 print(json_data[song_success_count])
#                 print('#############################')
#                 print(song_score)

                # song_id song_name
                song_list.append([
                    song_id,
                    json_data[song_success_count]["song"]["name"]
                ])
                # song_id ranklist_id song_score
                song_rank_list.append([
                    song_id,
                    ranklist_id,
                    song_score
                ])
                # multiple artists
                artist_count = 0
                for artist in json_data[song_success_count]["song"]["ar"]:
                    # artist_id artist_name
                    artist_list.append([
                        artist["id"],
                        artist["name"]
                    ])
                    # artist_id song_id sort
                    artist_song_list.append([
                        artist["id"],
                        song_id,
                        artist_count
                    ])
                    artist_count += 1
                # user_id song_id ranklist_all/week_score
                user_song_list.append([
                    user_id,
                    song_id,
                    song_score
                ])
                song_success_count += 1
        except Exception as e:
            logger.error("get_user_ranklist_songs parse failed",
                         "user_id:{},rank_type:{},error_type:{},song_success_count:{},error:{}".
                         format(user_id, rank_type, song_success_count, type(e), e))
            return False, None
        
        print("get_user_ranklist_songs success",
                    "user_id:{},rank_type:{},song_success_count:{}".format(user_id, rank_type, song_success_count))
        return (rank_list, song_list, artist_list, user_rank_list, song_rank_list, artist_song_list)

In [36]:
print(get_user_ranklist_songs())

b'{"weekData":[{"playCount":0,"score":100,"song":{"name":"I Need a Doctor","id":2416737,"pst":0,"t":0,"ar":[{"id":74801,"name":"Skylar Grey","tns":[],"alias":[]},{"id":32665,"name":"Eminem","tns":[],"alias":[]},{"id":53243,"name":"Dr. Dre","tns":[],"alias":[]}],"alia":[],"pop":100.0,"st":0,"rt":"","fee":8,"v":25,"crbt":null,"cf":"","al":{"id":243867,"name":"I Need a Doctor","picUrl":"http://p1.music.126.net/TWtqeOoMCALkTujxyjr7nQ==/109951163221173146.jpg","tns":[],"pic_str":"109951163221173146","pic":109951163221173146},"dt":283000,"h":{"br":320000,"fid":0,"size":11354096,"vd":-22000.0},"m":{"br":192000,"fid":0,"size":6812551,"vd":-19400.0},"l":{"br":128000,"fid":0,"size":4541779,"vd":-17700.0},"a":null,"cd":"1","no":1,"rtUrl":null,"ftype":0,"rtUrls":[],"djId":0,"copyright":1,"s_id":0,"mark":270336,"mst":9,"cp":7003,"mv":29956,"rtype":0,"rurl":null,"publishTime":1296489600007,"privilege":{"id":2416737,"fee":0,"payed":0,"st":-100,"pl":0,"dl":0,"sp":7,"cp":1,"subp":1,"cs":false,"maxbr":3

In [24]:
demo = get_user_ranklist_songs()

{'params': 'e1DsKnuam7sZ3apwcfNbCxWpWYFJQrdApufMAPeU/r8zhiffZL3ALkG2b5I8IEX8', 'encSecKey': '257348aecb5e556c066de214e531faadd1c55d814f9be95fd06d6bff9f4c7a41f831f6394d5a3fd2e3881736d94a02ca919d952872e7d0a50ebfa1769a7a62d512f5f1ca21aec60bc3819a9c3ffca5eca9a0dba6d6f7249b06f5965ecfff3695b54e1c28f3f624750ed39e7de08fc8493242e26dbc4484a01c76f739e135637c'}
get_request_data success url:https://music.163.com/weapi/v1/play/record ,first_param:{uid:"1493673206",type:"1"},proxies:None
get_user_ranklist_songs success user_id:1493673206,rank_type:1,song_success_count:100


In [25]:
print(demo[5])

[[74801, 2416737, 0], [32665, 2416737, 1], [53243, 2416737, 2], [32665, 17572558, 0], [780003, 480426313, 0], [45129, 480426313, 1], [3685, 432509483, 0], [33162096, 1388791055, 0], [551689, 1388791055, 1], [12707784, 1388791055, 2], [2116, 67787, 0], [32665, 545807814, 0], [1060155, 545807814, 1], [35334, 5103312, 0], [46489, 5103312, 1], [32665, 5052319, 0], [32665, 5052317, 0], [69270, 487590417, 0], [1048051, 487590417, 1], [37962, 487587444, 0], [69270, 487587444, 1], [13919979, 487587444, 2], [69270, 461347784, 0], [33184, 460043704, 0], [1181741, 409647285, 0], [44263, 409647285, 1], [98105, 409647285, 2], [53283, 409031076, 0], [1023337, 409031076, 1], [12353034, 409031076, 2], [91787, 4054121, 0], [9940, 402070928, 0], [72724, 3313334, 0], [29968, 3313334, 1], [10204, 317355, 0], [740216, 29544142, 0], [32665, 28558303, 0], [32665, 28411569, 0], [32665, 27906330, 0], [45236, 27713940, 0], [69491, 27713940, 1], [27604, 27713940, 2], [44267, 26965405, 0], [32665, 26965405, 1], [

In [26]:
ranklist=get_user_ranklist_songs()
song_ranks= ranklist[4]
for i in range(len(song_ranks)):
    song_ranks[i].insert(0,ranklist[1][i][1])

print("[song_name, song_id, user_id(ro means total record), score(represent freuency rate 0% ~100%)]")
for i in song_ranks:
    print(i)

{'params': 'e1DsKnuam7sZ3apwcfNbCxWpWYFJQrdApufMAPeU/r8zhiffZL3ALkG2b5I8IEX8', 'encSecKey': '257348aecb5e556c066de214e531faadd1c55d814f9be95fd06d6bff9f4c7a41f831f6394d5a3fd2e3881736d94a02ca919d952872e7d0a50ebfa1769a7a62d512f5f1ca21aec60bc3819a9c3ffca5eca9a0dba6d6f7249b06f5965ecfff3695b54e1c28f3f624750ed39e7de08fc8493242e26dbc4484a01c76f739e135637c'}
get_request_data success url:https://music.163.com/weapi/v1/play/record ,first_param:{uid:"1493673206",type:"1"},proxies:None
get_user_ranklist_songs success user_id:1493673206,rank_type:1,song_success_count:100
[song_name, song_id, user_id(ro means total record), score(represent freuency rate 0% ~100%)]
['I Need a Doctor', 2416737, '1493673206r1', 100]
['Mockingbird', 17572558, '1493673206r1', 100]
['There For You', 480426313, '1493673206r1', 75]
['全世界谁倾听你 ', 432509483, '1493673206r1', 75]
['Senorita', 1388791055, '1493673206r1', 75]
['预感', 67787, '1493673206r1', 50]
['Nowhere Fast (Extended Version)', 545807814, '1493673206r1', 50]
['Empi